<h1><center>Laboratorio 6: La desperación de Mr. Lepin 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Constanza Peña

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Kevin Pinochet Hernández


### **Link de repositorio de GitHub:** `https://github.com/Keviinplz/MDS7202-1`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripcción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines.
- Aplicar Clusters sobre un conjunto de datos.

## Reglas:

- Fecha de entrega: 12/11/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Comprender y aprovechar las ventajas que nos ofrece la librería `pandas` con respecto a trabajar en Python 'puro'.
- Crear nuevas características para entrenar un modelo de clustering.
- Comprender como aplicar pipelines de Scikit-Learn para generar procesos más limpios.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import HTML

import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

In [2]:
import os
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')
    path = os.path.join(os.getcwd(), 'data')

def get_file(filename: str) -> str:
    """Obtiene la ruta exacta al archivo especificado"""
    return os.path.join(path, filename)

Ignorando conexión drive-colab


<p align="center"><iframe width="560" height="315" src="https://www.youtube.com/embed/MW4d8_LLYp8?rel=0&amp;controls=0&amp;showinfo=0" frameborder="1" allowfullscreen></iframe></p>

# Segmentación de Clientes en Tienda de Retail 🛍️

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

## 1.1 Cargar Dataset

Mr. Lepin, en una nueva reunión, le cuenta a ud y su equipo que los resultados derivados del análisis exploratorio de dato presentaron una gran utilidad para la empresa y que tiene un gran entusiasmo por continuar trabajando con ustedes.
Es por esto, que Mr. Lepin les pide que cargue y visualicen algunas de las filas que componen el Dataset.
A continuación un extracto de lo parlamentado en la reunión:

    - Usted: Es un gran logro para nuestro equipo que usted haya encontrado excelente el EDA. ¿Qué tiene en mente ahora?
    - Mr. Lepin: Resulta que hace algún tiempo, mientras tomaba un mojito en una reunión de gerentes en Panamá, oí a un *chato* acerca de **LRMFP**, que es un modelo que permite personificar a los clientes a través de la farbicación de distintos atributos que describen a los clientes. Lo encontré es-tu-pendo ñatito. 
    - Usted: Ehh bueno. Investigaremos acerca de este modelo y veremos lo que podemos hacer.

Por ende, su siguiente tarea es calcular **LRMFP** sobre cada cliente y luego hacer un análisis de las características generadas. Para esto, el área de ventas les entrega un nuevo archivo llamado `online_retail_II_cleaned.pickle`, quien posee los datos del DataFrame original limpios y listos para obtener las características solicitadas por Mr. Lepin.

In [3]:
df_retail: pd.DataFrame = pd.read_pickle(get_file("online_retail_II_cleaned.pickle"))
df_retail = df_retail.astype(
    {
        "Invoice": "category",
        "StockCode": "category",
        "Description": "category",
        "Description": str,
        "Customer ID": "category",
        "Country": "category"
    }
)
df_retail.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


## 1.2 Creación de nuevas Caracteristicas [2 Puntos] 

Como ya se les comento, Mr. Lepin esta interesado en obtener las características **LRMFP**, para esto les señala que estas características se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.
- **Recency (R)**: Indica la actualidad de la interacción de un cliente con la empresa, y da información sobre la tendencia a repetir la compra. Se define como: 
$$Recency(n)=\dfrac{1}{n} \sum^n_{i=1} date\_diff(t_{fecha final}, t_{m-i+1})$$

 Donde $date\_diff$ representa la diferencia en días entre la fecha de finalización del periodo de observación ($t_{fecha final}$), y la fecha de una visita del cliente cercana a $t_{fecha final}$, $t_{m-i+1}; t_{m}$ es la última visita del cliente; y n es el número de visitas recientes del cliente consideradas.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente. 

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.
 

$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**Nota:** Para la $fechafinal$ utilice la fecha máxima del dataset más 1 día.

**Ejemplo de Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [5]:
from typing import Callable, List
from functools import reduce

Transformable = Callable[[pd.DataFrame, pd.DataFrame], pd.DataFrame]

def date_diff(max_date: datetime, min_date: datetime) -> int:
    """Returns the difference in days between two dates"""

    return (max_date - min_date).days

def compose(*functions: Transformable) -> Transformable:
    """Returns a new function that applies each transformable functions in order"""

    return reduce(lambda f, g: lambda df, init: g(df, f(df, init)), functions)

def length_customer(df: pd.DataFrame, customers: pd.DataFrame) -> pd.DataFrame:
    """Returns a new dataframe calculate max time interval in days for each client in dataframe"""

    max_dt: pd.Series = df.groupby("Customer ID")["InvoiceDate"].max()
    min_dt: pd.Series = df.groupby("Customer ID")["InvoiceDate"].min()

    # mypy doesn't recognize the days attribute of dt.
    interval: pd.Series = (max_dt - min_dt).dt.days # type: ignore 

    interval.rename("Length", inplace=True)
    return customers.join(interval, on="Customer ID")

def recency_customer(df: pd.DataFrame, customers: pd.DataFrame) -> pd.DataFrame:
    """
    Returns a new dataframe calculate recency in days for each client in dataframe
    The last invoice date of each client will be considerated, equivalent to set 
    n = 1 according to the paper.
    """

    end_date: datetime = df["InvoiceDate"].max() + timedelta(days=1)
    last_date_client_interaction: datetime = df.groupby("Customer ID")["InvoiceDate"].max()
    recency: pd.Series = (end_date - last_date_client_interaction).dt.days # type: ignore

    recency.rename("Recency", inplace=True)
    return customers.join(recency, on="Customer ID")

def monetary_customer(df: pd.DataFrame, customers: pd.DataFrame) -> pd.DataFrame:
    """Returns a new dataframe calculate monetary value for each client in dataframe"""
    monetary: pd.Series = df.groupby("Customer ID")["Price"].mean()

    monetary.rename("Monetary", inplace=True)
    return customers.join(monetary, on="Customer ID")

def frequency_customer(df: pd.DataFrame, customers: pd.DataFrame) -> pd.DataFrame:
    """Returns a new dataframe calculate frequency for each client in dataframe"""
    frequency: pd.Series = df.groupby(["Customer ID"])["InvoiceDate"].nunique()

    frequency.rename("Frequency", inplace=True)
    return customers.join(frequency, on="Customer ID")

def periodicity_customer(df: pd.DataFrame, customers: pd.DataFrame) -> pd.DataFrame:
    """Returns a new dataframe calculate periodicity for each client in dataframe"""
    # Must to be an absolute better way to do this, but I don't know how to do it.
    # I'm using a very hacky way.
    def _reduce_acumm(x: datetime, y: datetime, ivts: List[str]) -> datetime:
        ivts.append((y - x).days)
        return y

    def periodicity(gb: pd.DataFrame) -> pd.DataFrame:
        if len(gb["InvoiceDate"]) > 1:
            ivts: List[int] = []
            reduce(lambda x, y: _reduce_acumm(x, y, ivts), gb["InvoiceDate"])
            return np.std(np.array(ivts))
        else:
            return 0

    customer_invoice_date: pd.DataFrame = df[["Customer ID", "InvoiceDate"]].copy()
    customer_invoice_date["InvoiceDate"] = customer_invoice_date["InvoiceDate"].dt.date
    customer_invoice_date.drop_duplicates(inplace=True)
    customer_invoice_date.sort_values(by=["Customer ID", "InvoiceDate"], inplace=True)
    p: pd.Series = customer_invoice_date.groupby("Customer ID").apply(periodicity)

    p.rename("Periodicity", inplace=True)
    return customers.join(p, on="Customer ID")


def custom_features(df: pd.DataFrame) -> pd.DataFrame:
    """Returns a new dataframe applying a LRMFP study on given dataframe""" 

    result: pd.DataFrame = pd.DataFrame()
    result["Customer ID"] = df["Customer ID"].unique()
    p = compose(
        length_customer,
        recency_customer,
        monetary_customer,
        frequency_customer,
        periodicity_customer
    )
    return p(df, result)

In [6]:
new_df: pd.DataFrame = custom_features(df_retail)

**Resúesta de Pauta:**

## 1.3 Pipelines 👷

Finalmente *Don Mora* le pregunta si seria posible realizar un pipeline para realizar una segmentación de los clientes con los nuevos datos generados, a lo que usted responde que **sí** y propone la utilización de k-means para la segmentación.

A continuación siga los pasos requeridos para obtener la segmentación de clientes.

### 1.3.1 Estandarizar Caracteristicas [0.5 puntos]

Construya una clase llamada ``MinMax()`` utilizando ``BaseEstimator`` y ``TransformerMixin`` para realizar una transformación de cada una de las columnas de un DataFrame utilizando ``ColumnTransformer()`` más tarde (tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer)).


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$

Con esto buscamos que los valores que componen a las columnas se muevan en el rango de valores $[0, 1]$.

**Respuesta:**

In [7]:
from __future__ import annotations
# I'm using 'annotations' because fit method on MinMax class
# returns an MinMax object, but MinMax is not defined yet.
# So, in the future, to avoid this, python will do annotations strings in runtime.

class MinMax(BaseEstimator, TransformerMixin):
    def fit(self, X: pd.DataFrame, y: pd.Series = None) -> MinMax:
        return self
    
    def transform(self, x: pd.DataFrame) -> pd.DataFrame:
        return (x - x.min()) / (x.max() - x.min())

### 1.3.2 `T-SNA` Pipeline [1.0 puntos]

Para comenzar introduciéndose en el uso de pipeline, decide probar realizando un pipeline enfocado en la reducción de dimensionalidad y así hacer no decepcionar a Mr. Lepin con la clusterización del modelo. 

Configure un pipeline utilizando el algoritmo `T-SNE` sobre los datos **LRMFP**, donde, para la realización del pipeline considera los siguientes pasos:

1. Como primer paso obtenga las características **LRMFP** desde el DataFrame ``df_retail_II_cleaned.pickle`` utilizando la función ``custom_features`` creada anteriormente, junto a ``FunctionTransformer()``. Considere esto como el primer paso de su pipeline.
2. En segundo lugar usando ``ColumnTransformer()`` aplique el MinxMax scaler creado por usted sobre todas las columnas generadas en el paso anterior. 
3. Finalmente, aplique un último paso donde obtiene las 2 componentes más relevantes utilizando el algoritmo T-sne de sckit-learn.

Tras aplicar las transformaciones sobre el dataset **LRMFP**, gráfique las componentes obtenidas en la reducción de dimensionalidad.

**Respuesta:**

In [9]:
# Bergel always set random seed at 42, which I thought was funny because 42 is the meaning of life.
# So I'm using 42 as well.
seed: int = 42

pipe: Pipeline = Pipeline([
    ("to_lrmfp", FunctionTransformer(custom_features)),
    ("minmax", ColumnTransformer(
        transformers=[
            ("minmax", MinMax(), ["Length", "Recency", "Monetary", "Frequency", "Periodicity"]
    )])),
    ("T-SNE", TSNE(n_components=2, random_state=seed))
])
transformed: np.ndarray = pipe.fit_transform(df_retail)
transformed

C:\Users\Kevii\AppData\Local\pypoetry\Cache\virtualenvs\lab6-gcaqUmwU-py3.9\lib\site-packages\sklearn\manifold\_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
C:\Users\Kevii\AppData\Local\pypoetry\Cache\virtualenvs\lab6-gcaqUmwU-py3.9\lib\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


array([[ 17.318815 , -24.377306 ],
       [  3.4987564,  69.83457  ],
       [-12.6269455,  38.682743 ],
       ...,
       [-61.344917 ,   5.374823 ],
       [-61.347004 ,   5.3748417],
       [-61.344337 ,   5.374798 ]], dtype=float32)

In [17]:
# Plotting with plotly
new_df["x_tsne"] = transformed[:, 0]
new_df["y_tsne"] = transformed[:, 1]
px.scatter(new_df, x="x_tsne", y="y_tsne", color="Length", title="Reducción sobre Length")

In [18]:
px.scatter(new_df, x="x_tsne", y="y_tsne", color="Recency", title="Reducción sobre Recency")

In [20]:
# Presencia de outliers entorpecen la visualización
px.scatter(new_df, x="x_tsne", y="y_tsne", color="Monetary", title="Reducción sobre Monetary")

In [21]:
# Outliers entorpesen la visualización
px.scatter(new_df, x="x_tsne", y="y_tsne", color="Frequency", title="Reducción sobre Frequency")

In [23]:
px.scatter(new_df, x="x_tsne", y="y_tsne", color="Periodicity", title="Reducción sobre Periodicity")

### 1.3.3 Clustering

#### 1.3.3.1 Método del Codo [1 puntos]

Utilizando la clase creada para escalamiento, aplique el método del codo para visualizar cual es el número de clusters que mejor se ajustan a los datos. Realice esto utilizando el algoritmo K-means dentro de un pipeline para un $k \in [1,20]$, donde k representa el número de clusters del k-means. Para la realización de esta sección y la próxima (1.3.3.2), considere los mismos pasos utilizados para el t-sne, pero permutando el algoritmo de reducción de dimensionalidad por k-means.

A través del grafico obtenido, comente y justifique que valor de k escogería para realizar el k-means.

**Respuesta:**

In [29]:
def make_k_means_pipeline(n_clusters: int) -> Pipeline:
    """Returns a k-means pipeline with Function and Column transformer applies over data"""
    return Pipeline([
        ("to_lrmfp", FunctionTransformer(custom_features)),
        ("minmax", ColumnTransformer(
            transformers=[
                ("minmax", MinMax(), ["Length", "Recency", "Monetary", "Frequency", "Periodicity"]
    )])),
        ("kmeans", KMeans(n_clusters=n_clusters, random_state=seed))
    ])

pipes: List[Pipeline] = [make_k_means_pipeline(n_clusters) for n_clusters in range(1, 20)]
inertias: List[List[int, float]] = []

for i_cluster, pipe in enumerate(pipes):
    pipe.fit(df_retail)
    inertias.append([i_cluster + 1, pipe.named_steps["kmeans"].inertia_])

df_inertias: pd.DataFrame = pd.DataFrame(inertias, columns=["Clusters", "Inertia"])
px.line(df_inertias, x="Clusters", y="Inertia", title="Método del Codo con K-Means")

Dado el gráfico anterior, se puede apreciar el corte en el k-means de cluster = 4, por lo que escojemos ese para realizar el método

#### 1.3.3.2 Segmentación de Clientes con K-Means 🎁 [1 punto]

En base a la elección de k realizada en la sección anterior, utilice este valor escogido y entrene un modelo de K-means utilizando el mismo pipeline de scikit-learn utilizado anteriormente.

Una vez ajustado los datos, genere una tabla con los promedios (o medianas) para cada uno de los atributos, agrupando estos por el clúster que pertenecen. ¿Es posible observar agrupaciones coherentes?, ¿Qué tipo de clientes posee el retail?, Justifique su respuesta y no decepcione a Mr. Lepin.


**Respuesta:**

In [55]:
three_means_pipe: Pipeline = make_k_means_pipeline(4)
result: np.ndarray = three_means_pipe.fit_predict(df_retail)
lrmfp_df: pd.DataFrame = custom_features(df_retail)

In [56]:
lrmfp_df["Cluster"] = pd.Series(result)
lrmfp_df = lrmfp_df[["Length", "Recency", "Monetary", "Frequency", "Periodicity", "Cluster"]]

In [57]:
lrmfp_df.groupby("Cluster").mean()

,Length,Recency,Monetary,Frequency,Periodicity
Cluster,,,,,
0,18.157205,254.983624,26.955026,1.505459,1.309236
1,317.696191,24.184234,3.607880,10.079717,38.473113
2,17.443750,52.238281,8.098449,1.694531,1.134364
3,182.166835,66.735086,3.445114,4.298281,19.874571


Podemos observar que el primer cluster agrupa a los clientes que no han comprado durante mucho tiempo (Recency alto).

El segundo cluster, agrupa a los clientes que llevan comprando en la tienda durante mucho tiempo (Length alto).

El tercer y el cuarto cluster pareciera agrupar a clientes promedio, donde los terceros son menos frecuentes que los cuartos, sin embargo
estos últimos llevan más tiempo comprando

**Respuesta Esperada:**

|         | Length  | Recency   | Frequency | Monetary | Periodicity |       |
|---------|---------|-----------|----------|-------------|-------|-------|
| Cluster |         |           |          |             |       |       |
|    0    |   258.8 |      45.2 |     76.1 |      1107.7 | 107.6 |   449 |
|    1    |    76.1 |     217.6 |     45.5 |       791.7 |  14.1 |   466 |
|    2    |   368.5 |       4.8 |   2715.0 |    226621.6 |   4.2 |     4 |
|    3    |    85.3 |      45.7 |     65.8 |      1047.0 |  10.5 |   987 |
|    4    |   347.2 |      15.9 |   1658.0 |     35829.3 |   8.0 |    25 |
|    5    |   298.0 |      29.8 |    183.8 |      3639.9 |  32.0 |  1188 |

#### 1.3.3.3 Plot de K-Means 📈 [0.5 puntos]



Por último, Mr. Lepin, impaciente de no entender lo que usted intenta explicarle, le solicita que por favor muestre algún resultado "visual" de los grupos encontrados.

Para esto, grafique nuevamente las características encontradas usando `T-SNE` (no calcule de nuevo, simplemente utilice las proyecciones encontradas) y agregue las labels calculadas con kmeans como el argumento `color`.   

Comente: ¿Se separan bien los distintos clusters en la visualización?


**Respuesta:**

In [58]:
lrmfp_df["x_tsne"] = transformed[:, 0]
lrmfp_df["y_tsne"] = transformed[:, 1]

In [60]:
px.scatter(lrmfp_df, x="x_tsne", y="y_tsne", color="Cluster", title="K-Means sobre Dataset")

Se logra apreciar una separación semicoherente, el segundo con el último cluster están juntos debido a que
son clientes que llevan comprando durante harto tiempo, pero tienen diferencias en otros atributos.

El primer con el tercer cluster son totalmente distintos, sin embargo el método no logra separarlos del todo.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana y que **los días de atraso no se pueden utilizar para entregas de lab** solo para tareas. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>